## The first big project - Professionally You!
## And, Tool use.

## But first: introducing MailGun
I am using MailGun to send email notifications to my noreply email

- Go to https://www.mailgun.com/
- Signup for free
- create API Key using their SandBox Domain
- Add recipients in the recipients list. Recipients will need to agree to receive email notifications

In [ ]:
import os
from dotenv import load_dotenv
import requests
import json
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

load_dotenv(override=True)
openai_client = OpenAI()
gemini_openai_client = OpenAI(api_key=os.getenv("GEMINI_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
openai_model_name = "gpt-4o-mini"
gemini_model_name = "gemini-2.0-flash"

error_send_email= "Error sending email notification!"
success_send_email= "Email notification successful!"

In [70]:
# MailGun API to send email notifications
def send_mailgun_email_notification(subject, body):
    return requests.post(
        f"https://api.mailgun.net/v3/{os.getenv('MAIL_GUN_DOMAIN')}/messages",
        auth=("api", os.getenv('MAIL_GUN_API_KEY')),
        data={
            "from": os.getenv('MAIL_GUN_FROM_EMAIL'),
            "to": os.getenv('MAIL_GUN_TO_EMAIL'),
            "subject": subject,
            "text": body
        }
    )

In [ ]:
# Test MailGun API
send_mailgun_email_notification(
    subject="Hello VK Avatar",
    body="Hey Varun, Notebook Setup Job completed successfully!"
)

In [71]:
# Tool: Notify When User Wants to Get in Touch

def notify_user_interest(user_name, user_email, message):
    subject = f"[Contact Request] {user_name} wants to get in touch"
    body = f"""Hi Varun,
    
        You have a new contact request from your AI Agent Avatar website:
        
            Name: {user_name}
            Email: {user_email}
            Message:
            {message}
        
        — AI Avatar Bot"""

    try:
        send_mailgun_email_notification(subject, body)
    except Exception as e:
        print(f"[Contact Request] Error sending email: {e}")
        return error_send_email

    return success_send_email

In [72]:
# Tool: Notify When Agent Cannot Answer a Question

def notify_agent_failure(user_question, agent_response):
    subject = "[Unanswered Question] Agent Avatar could not respond"
    body = f"""Hi Varun,
        Your AI Agent Avatar couldn't confidently answer a user's question.
        
        User Question:
        {user_question}
        
        Agent's Attempted Response:
        {agent_response}
        
        You might want to update the knowledge base or manually follow up.
        
        — AI Avatar Bot"""

    try:
        send_mailgun_email_notification(subject, body)
    except Exception as e:
        print(f"[Contact Request] Error sending email: {e}")
        return error_send_email

    return success_send_email


In [73]:
# Tool JSON: Notify When a User Wants to Get in Touch

notify_user_interest_tool_json = {
  "name": "notify_user_interest",
  "description": "Send an email to me when a visitor wants to get in touch.",
  "parameters": {
    "type": "object",
    "properties": {
      "user_name": {
        "type": "string",
        "description": "The full name of the visitor trying to reach out."
      },
      "user_email": {
        "type": "string",
        "description": "The visitor's email address."
      },
      "message": {
        "type": "string",
        "description": "The message the visitor wants to send to Varun."
      }
    },
    "required": ["user_name", "user_email", "message"]
  }
}


In [74]:
# Tool JSON: Notify When Agent Cannot Answer a Question

notify_agent_failure_tool_json = {
  "name": "notify_agent_failure",
  "description": "Notify me that the agent could not answer a user's question.",
  "parameters": {
    "type": "object",
    "properties": {
      "user_question": {
        "type": "string",
        "description": "The question asked by the visitor."
      },
      "agent_response": {
        "type": "string",
        "description": "The agent's attempted or fallback response."
      }
    },
    "required": ["user_question", "agent_response"]
  }
}


In [91]:
# Create tool_spec array

tool_spec = [ 
    {"type": "function", "function": notify_user_interest_tool_json}, 
    {"type": "function", "function": notify_agent_failure_tool_json} 
]

In [ ]:
tool_spec

In [79]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

"""
Here’s what happens step-by-step:

    1. User sends a message in Chatbox.
    2. LLM/AI_Agent chooses a tool to call.
    3. We (Python Script) receive a message like:
    {
        "role": "assistant",
        "tool_calls": [
            {
                "id": "tool_call_id_xyz123",
                "function": {
                    "name": "notify_user_interest",
                    "arguments": "{ \"user_name\": \"John Doe\", \"user_email\": \"john@example.com\", \"message\": \"I want to connect.\" }"
                }
            }
        ]
    }

    4. We run the tool notify_user_interest() with the arguments { "user_name": "John Doe", "user_email": "john@example.com", "message": "I want to connect." }
    5. We return the result to the LLM
        {
            "role": "tool",
            "content": "Notification sent successfully.",
            "tool_call_id": "tool_call_id_xyz123"
        }

    6. LLM/AI_Agent sees the result and can continue with the conversation
"""

def handle_tool_calls_with_if_else(tool_calls):
    respond_to_agent_tool_calls = []

    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        tool_arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: Name: {tool_name}, Arguments: {tool_arguments}", flush=True)

        # THE BIG IF-ELSE!!!
        if tool_name == "notify_user_interest":
            result = notify_user_interest(**tool_arguments)
        elif tool_name == "notify_agent_failure":
            result = notify_agent_failure(**tool_arguments)
        else:
            unknown_tool_call = f"Unknown tool call: {tool_name}"
            print(unknown_tool_call, flush=True)
            result = unknown_tool_call 

        respond_to_agent_tool_calls.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id
        })

    return respond_to_agent_tool_calls

In [ ]:
# globals() usage as an alternative to IF-ELSE above for cleaner code
# globals() is a built-in Python function that returns a dictionary containing the current global namespace - 
# essentially all the variables, functions, and classes that are available in the current module (.py file).

"""
What globals() Includes:
- Module-level variables
- Module-level functions
- Module-level classes
- Built-in functions/variables
- Imported modules

What globals() Does NOT Include:
- Class-level variables (class attributes)
- Instance variables
- Local variables inside functions
- Variables inside classes
"""

globals()["notify_agent_failure"]("this is a really hard question", "I'm sorry, I can't answer that question.")

globals()["notify_user_interest"]("John Doe", "john@example.com", "I want to connect.")

In [81]:
def handle_tool_calls(tool_calls):
    """
    - Problem: 
        globals() includes built-in functions. This could be dangerous - LLM could call ANY function!

        below code:

        malicious_function = "os.system"  # Could delete files!
        if malicious_function in globals():
            globals()[malicious_function]("rm -rf /")  # DANGEROUS!

    - So, always make sure to allow only a specific list of tool functions to be called.
    """
    
    # This is a list of tool functions that are allowed to be called.
    allowed_tool_functions = ["notify_user_interest", "notify_agent_failure"]

    respond_to_agent_tool_calls = []

    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        tool_arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: Name: {tool_name}, Arguments: {tool_arguments}", flush=True)

        if tool_name in allowed_tool_functions and tool_name in globals():
            tool_function = globals()[tool_name]
            result = tool_function(**tool_arguments)
        else:
            unknown_tool_call = f"Unknown tool call: {tool_name}"
            print(unknown_tool_call, flush=True)
            result = unknown_tool_call

        respond_to_agent_tool_calls.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id
        })

    return respond_to_agent_tool_calls

In [82]:
full_name = "Varun Kakuste"
first_name = "Varun"

linkedIn_profile_text = ""

linkedIn_Profile_pdf_reader = PdfReader("me/My_LinkedIn_Profile.pdf")
for page in linkedIn_Profile_pdf_reader.pages:
    page_text = page.extract_text()
    if page_text:
        linkedIn_profile_text += page_text

with open("me/summary.txt", "r", encoding="utf-8") as summary_file:
    summary_text = summary_file.read()

In [ ]:
system_prompt_template = """
You are the professional AI Agent Avatar of {full_name}. You represent {first_name} on their personal website and respond on their behalf to visitors who may include hiring managers, investors, founders, recruiters, and other professionals.

You have access to the following structured information:
- LinkedIn Profile: {linkedIn_profile_text}
- Summary/Bio: {summary_text}

You are expected to respond in a tone that is:
- Professional
- Friendly
- Authentic to {first_name}'s voice
- Clear, concise, and context-aware

Your role is to provide high-quality, truthful, and helpful responses using only the above knowledge. Do not guess or fabricate details.

---

### TOOL USAGE INSTRUCTIONS

You have access to tools and may call them when appropriate:

**1. Tool: `notify_user_interest`**
- Use when the visitor expresses interest in contacting {first_name} (e.g., scheduling a call, asking how to connect, requesting to talk, etc.).
- Do not use this tool for casual compliments or generic remarks.

**2. Tool: `notify_agent_failure`**
- Use notify_agent_failure whenever you are unable to answer the user’s question, and the question appears to be personal, private, sensitive, or not covered in the available information.
- You must not guess or make up answers. If unsure, use this tool to alert {first_name} so he can follow up.
- In addition but **not limited to** below, if the user asks about:
    - Private career goals
    - Salary, family, politics, or religion
    - Deep technical details not in profile
    You should call the **notify_agent_failure** tool.
- Never make assumptions about {first_name}'s work authorization, immigration status, or other personal legal details. If a visitor asks about these topics, trigger the `notify_agent_failure` tool instead of responding directly.

After using a tool, wait for the tool response before continuing the conversation. Do not continue unless the result of the tool call is available.

---

### SAFETY & FALLBACK BEHAVIOR

- If you’re unsure of an answer, do **not fabricate** information. Instead, either politely decline to answer or trigger the `notify_agent_failure` tool.
- Do not provide medical, legal, financial, or personal advice unless it is already present in the LinkedIn or summary data.
- Avoid speculation about confidential topics like salary, political opinions, or personal relationships.

---

### FINAL GUIDELINES

- Never refer to yourself as "AI" or "Chatbot."
- Always speak in the first person as if you are {first_name}.
- Maintain trust, professionalism, and clarity in all interactions.

You are a capable, intelligent digital assistant version of {first_name}, designed to create meaningful, respectful, and accurate professional interactions.

"""

In [ ]:
system_prompt = system_prompt_template.format(
    full_name=full_name,
    first_name=first_name,
    linkedIn_profile_text=linkedIn_profile_text,
    summary_text=summary_text
)

print(system_prompt)

In [115]:
# Test the agent with a simple question for openai

# messages_to_agent = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "Are you a musician?"}]

# response = openai_client.chat.completions.create(
#     model=openai_model_name,
#     messages=messages_to_agent,
#     tools=tool_spec
# )

# reply = response.choices[0].message.content
# print(reply)

In [ ]:
# # Test the agent with a simple question for gemini

# messages_to_agent = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "Do you require Visa Sponsorship?"}]

# response = gemini_openai_client.chat.completions.create(
#     model=gemini_model_name,
#     messages=messages_to_agent,
#     tools=tool_spec
# )

# print(response)

# reply = response.choices[0].message.content
# print(reply)

In [ ]:
# Test the agent with a simple question for OpenAI model (gpt-4o-mini)

def chat_with_agent_openai(user_prompt, history):
    messages_to_agent = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": user_prompt}]
    is_done = False
    while not is_done:
        response = openai_client.chat.completions.create(
            model=openai_model_name,
            messages=messages_to_agent,
            tools=tool_spec
        )

        # If the LLM/AI_Agent has finished its task, it will return a finish_reason object.
        finish_reason = response.choices[0].finish_reason
        if finish_reason == "tool_calls":
            # We get the message object from the response
            message = response.choices[0].message
            # If the LLM/AI_Agent needs to call a tool, it will return a tool_calls object.
            tool_calls = message.tool_calls
            # We run the tool calls
            tool_results = handle_tool_calls(tool_calls)
            # We add the tool results to the messages_to_agent list
            messages_to_agent.append(message)
            messages_to_agent.extend(tool_results)
        else:
            is_done = True

    agent_reply = response.choices[0].message.content
    print(agent_reply)
    messages_to_agent.append({"role": "assistant", "content": agent_reply})
    return agent_reply

In [ ]:
gr.ChatInterface(chat_with_agent_openai, type="messages").launch()

In [117]:
# Test the agent with a simple question for Gemini model (gemini-2.0-flash)

def chat_with_agent_gemini(user_prompt, history):
    messages_to_agent = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": user_prompt}]
    is_done = False
    while not is_done:
        response = gemini_openai_client.chat.completions.create(
            model=gemini_model_name,
            messages=messages_to_agent,
            tools=tool_spec
        )

        # If the LLM/AI_Agent has finished its task, it will return a finish_reason object.
        finish_reason = response.choices[0].finish_reason
        if finish_reason == "tool_calls":
            # We get the message object from the response
            message = response.choices[0].message
            # If the LLM/AI_Agent needs to call a tool, it will return a tool_calls object.
            tool_calls = message.tool_calls
            # We run the tool calls
            tool_results = handle_tool_calls(tool_calls)
            # We add the tool results to the messages_to_agent list
            messages_to_agent.append(message)
            messages_to_agent.extend(tool_results)
        else:
            is_done = True

    agent_reply = response.choices[0].message.content
    print(agent_reply)
    messages_to_agent.append({"role": "assistant", "content": agent_reply})
    return agent_reply

In [ ]:
gr.ChatInterface(chat_with_agent_gemini, type="messages").launch()